In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/ysal 2nd/'

In [ ]:
%pip install ultralytics

In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from ultralytics import YOLO
import math
import os

In [ ]:
model = YOLO('yolov8n.pt')

In [ ]:
os.chdir(path+"image/")

In [ ]:
baseline = cv2.imread(path + 'image/baseball.jpg')

In [ ]:
base_circle = np.array([
        [715,630],
        [715,1040],
        [510, 835], # Lower left
        [920, 835], # Lower right
        ],np.float32)

In [ ]:
base_cpy = baseline.copy()

In [ ]:
for i in base_circle:
  x, y = i
  x = int(round(x))
  y = int(round(y))
  base_cpy = cv2.circle(base_cpy, (x,y), radius=10, color=(0, 0, 255), thickness=-1)

cv2_imshow(base_cpy)

In [ ]:
def detect_white(img):
    white_range = ([0, 200, 0], [255, 255, 255])
    lower = np.array(white_range[0], dtype = "uint8")
    upper = np.array(white_range[1], dtype = "uint8")
    cvt = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = cv2.inRange(cvt, lowerb = lower, upperb = upper)
    result = cv2.bitwise_and(cvt, cvt, mask = mask)
    #cv2_imshow(result)
    portion = len(result[result!=0])/ (result.shape[0]*result.shape[1])
    return portion

def detect_non_white(img):
    non_white_range = ([0,0,0],[230, 230, 230])
    lower = np.array(non_white_range[0], dtype = "uint8")
    upper = np.array(non_white_range[1], dtype = "uint8")
    cvt = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = cv2.inRange(cvt, lowerb = lower, upperb = upper)
    result = cv2.bitwise_and(cvt, cvt, mask = mask)
    #cv2_imshow(result)
    portion = len(result[result!=0])/ (result.shape[0]*result.shape[1])
    return portion

def check_bbox_overlap(bbox1, bbox2):
    x1_min, y1_min, x1_max, y1_max = bbox1
    x2_min, y2_min, x2_max, y2_max = bbox2

    # 바운딩 박스가 겹치는지 확인
    overlap = not (x1_max < x2_min or x1_min > x2_max or y1_max < y2_min or y1_min > y2_max)

    # bbox1이 bbox2에 포함되는지 확인
    contained = x1_min >= x2_min and y1_min >= y2_min and x1_max <= x2_max and y1_max <= y2_max

    return overlap, contained

In [ ]:
def check_overlap(lst, check_lst) :

  overlap_tf = False

  if len(lst) != 0:
    for i in range(len(lst)) :

      overlap , contain = check_bbox_overlap(lst[i],check_lst)

      if ((overlap == True) or (contain ==True)) :
        overlap_tf = overlap

  return overlap_tf

In [ ]:
def detect_player(img) :

  white_pants = []
  white_player = []
  non_white_player = []

  results = model(img, conf=0.05)  # results list

  for result in results:

    boxes = result.boxes.cpu().numpy()
    for i, box in enumerate(boxes):

        if (box.cls[0] != 0 ) :
          continue

        r = box.xyxy[0].astype(int)

        crop = img[r[1]:r[3], r[0]:r[2]]
        top = img[r[1]: r[1] + int((r[3]-r[1])/3*1) ,r[0]:r[2]]
        bottom = img[r[1] + int((r[3]-r[1])/2) : r[3] , r[0] : r[2]]

        bottom_white = detect_white(bottom)


        if (bottom_white > 0.1) :

          tf = check_overlap(white_pants,r)

          if (tf == False) :
            white_pants.append(r)

            top_white = detect_white(top)

            if (top_white > 0.15) :
              white_player.append(r)
              #print("white")
              #cv2_imshow(crop)

            else :
              non_white_player.append(r)
              #print("non_white")
              #cv2_imshow(crop)

  # print("white")
  # print(white_player)

  # print("non_white")
  # print(non_white_player)

  return white_pants, white_player, non_white_player

In [ ]:
def check_pitcher(lst, midwidth, midheight) :

  center = [(item[0] + (item[2] - item[0])/2,item[3]) for item in lst]

  min_distance = float('inf')
  index = -1
  closest_coordinate = None

  for i,coordinate in enumerate(center):
    distance = math.sqrt((coordinate[0] - midwidth) ** 2 + (coordinate[1] - midheight) ** 2)
    if distance < 100:
      if distance < min_distance:
        index = i
        min_distance = distance
        closest_coordinate = coordinate

  return index, closest_coordinate

In [ ]:
def check_in_list(check_list, check_value):
    for item in check_list:
        if np.array_equal(item, check_value):
            return True
    return False

def defensor(index, white_pants, white_player, non_white_player, img) :

  defense_list = []
  pitcher_coor = white_pants[index]

  if check_in_list(white_player, pitcher_coor) :
    defense_list = white_player
    defense_list = [sublist for sublist in defense_list if not np.array_equal(sublist, pitcher_coor)]


  else :
    defense_list = non_white_player
    defense_list = [sublist for sublist in defense_list if not np.array_equal(sublist, pitcher_coor)]

  #for i in range(len(defense_list)) :
  #  cv2.rectangle(img, (defense_list[i][0], defense_list[i][1]), (defense_list[i][2], defense_list[i][3]), (0, 0 , 255), 2)  # 두께는 2로 가정

  #cv2_imshow(img)

  return defense_list

In [ ]:
def find_white_lines(img, extension_length):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # 가우시안 블러를 적용합니다.
  blurred = cv2.GaussianBlur(gray, (5, 5), 0)

  # Canny 엣지 검출을 수행합니다.
  edges = cv2.Canny(blurred, 50, 150)

  # Hough 선 변환을 적용합니다.
  lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100)
  extended_lines = []

  for line in lines:
      x1, y1, x2, y2 = line[0]
      length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
      thickness = 1  # Set the thickness for thin lines
      if length > 10 :  # Additional conditions for length and line color
          angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi

          if (15<abs(angle)<30) :

            extended_x1 = int(x1 - extension_length * np.cos(np.radians(angle)))
            extended_y1 = int(y1 - extension_length * np.sin(np.radians(angle)))
            extended_x2 = int(x2 + extension_length * np.cos(np.radians(angle)))
            extended_y2 = int(y2 + extension_length * np.sin(np.radians(angle)))
            #cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), thickness)
            cv2.line(img, (extended_x1, extended_y1), (extended_x2, extended_y2), (0, 255, 0), thickness)
            extended_lines.append( [extended_x1, extended_y1, extended_x2, extended_y2])

            #print(f"Angle: {angle} degrees")

  #cv2_imshow(img)

  return lines, extended_lines

In [ ]:
def remove_similar_lines(lines, img, threshold=0.1):
    result = []
    threshold_slope = threshold  # 기울기의 차이가 이 값 이하면 유사한 라인으로 간주

    def slope(line):
        x1, y1, x2, y2 = line
        if x2 - x1 == 0:
            return float('inf')  # 수직 라인의 경우 무한대의 기울기를 갖음
        else:
            return (y2 - y1) / (x2 - x1)

    for i, line1 in enumerate(lines):
        is_similar = False
        for j, line2 in enumerate(result):
            # 기울기의 차이가 일정 값 이하인 경우 유사한 라인으로 간주
            if abs(slope(line1) - slope(line2)) < threshold_slope:
                is_similar = True
                break
        if not is_similar:
            result.append(line1)

    return result

In [ ]:
def find_intersections(lines):
    intersections = []
    for i in range(len(lines)):
        for j in range(i + 1, len(lines)):
            line1 = lines[i]
            line2 = lines[j]
            x1, y1, x2, y2 = line1
            x3, y3, x4, y4 = line2

            # 계산을 편하게 하기 위해 각 선분을 직선의 방정식으로 나타냅니다.
            a1 = y2 - y1
            b1 = x1 - x2
            c1 = x2 * y1 - x1 * y2

            a2 = y4 - y3
            b2 = x3 - x4
            c2 = x4 * y3 - x3 * y4

            # 두 직선의 교점을 계산합니다.
            determinant = a1 * b2 - a2 * b1
            if determinant != 0:  # 직선이 평행하지 않을 경우에만 계산
                intersection_x = (b1 * c2 - b2 * c1) / determinant
                intersection_y = (a2 * c1 - a1 * c2) / determinant

                # 찾은 교점을 리스트에 추가합니다.
                intersections.append((int(intersection_x), int(intersection_y)))

    return intersections

In [ ]:
def find_3_base(intersections, pitcher_cor,img):
  base_home = tuple((0,10000))
  base_1st = tuple((10000,0))
  base_3rd = tuple((-1000,0))

  for i in range(len(intersections)) :

    if ((pitcher_cor[1] - 20) <  intersections[i][1]  < (pitcher_cor[1] + 20)) :

      if (((intersections[i][0] - pitcher_cor[0]) < 0 ) and ( -int(img.shape[1] * 0.25) > (intersections[i][0] - pitcher_cor[0]) > (base_3rd[0] -  pitcher_cor[0]))):
        base_3rd = intersections[i]

      elif (((intersections[i][0] - pitcher_cor[0]) > 0 ) and (int(img.shape[1] * 0.25) < (intersections[i][0] - pitcher_cor[0]) < (base_1st[0] -  pitcher_cor[0]))):
        base_1st = intersections[i]

    elif ((pitcher_cor[0] - 20) <  intersections[i][0]  < (pitcher_cor[0] + 20)) :
        if (((intersections[i][1] - pitcher_cor[1]) > 0 )  and (int(img.shape[1] * 0.1) < (intersections[i][1] - pitcher_cor[1]) < (base_home[1]- pitcher_cor[1]))):
          base_home = intersections[i]

  return base_home, base_1st, base_3rd

In [ ]:
def real_defense_coordinate(random_coordinate):
  x, y = random_coordinate[0], random_coordinate[1]
  scale = 19.395 / 205
  real_coordinate = [(x - 715) * scale, (1040 - y) * scale]
  if (real_coordinate[1] <= abs(real_coordinate[0]))  or (real_coordinate[1] > 70):
    return False , [0,0]

  else:
    pass

  return True , real_coordinate

In [ ]:
def main(img, extension_length, baseline, base_circle) :

  midwidth, midheight = int(img.shape[1]/2) , int(img.shape[0]/2)
  white_pants , white_player, non_white_player = detect_player(img)
  index, pitcher_cor = check_pitcher(white_pants, midwidth, midheight)

  if pitcher_cor is None :
    print("Yolo가 사람 인식을 제대로 수행하지 못했습니다.")
    return  False, None, None

  else :

    defense_list = defensor(index, white_pants, white_player, non_white_player, img)
    print(defense_list)
    if len(defense_list) < 4 :
      print("Yolo가 사람 인식을 제대로 수행하지 못했습니다.")
      return  False, None, None


  lines, extended_lines = find_white_lines(img,extension_length)
  unique_lines = remove_similar_lines(extended_lines, img, 0.1)
  unique_lines.append([0,int(pitcher_cor[1]), int(img.shape[1]), int(pitcher_cor[1])])
  unique_lines.append([int(pitcher_cor[0]), 0 , int(pitcher_cor[0]),  int(cnt.shape[0])])


  for i in range(len(unique_lines)) :
    cv2.line(cnt, (unique_lines[i][0], unique_lines[i][1]), (unique_lines[i][2], unique_lines[i][3]), (255, 255, 0), 3)

  for i in range(len(defense_list)) :
   cv2.rectangle(img, (defense_list[i][0], defense_list[i][1]), (defense_list[i][2], defense_list[i][3]), (0, 0 , 255), 2)  # 두께는 2로 가정

  #cv2_imshow(img)


  intersections = find_intersections(unique_lines)
  base_home, base_1, base_3 = find_3_base(intersections, pitcher_cor,img)
  base_2 = tuple((pitcher_cor[0] , int(base_home[1] - (base_home[1] - pitcher_cor[1])*1.55 )))
  base_coor = [base_2,base_home,base_3,base_1]

  for i in range(len(base_coor)) :
    cv2.circle(img, (int(base_coor[i][0]),int(base_coor[i][1])), radius=10, color=(255, 0, 0), thickness=3)

  cv2_imshow(img)

  base_coor = np.array(base_coor, dtype=np.float32)
  transformation_matrix = cv2.getPerspectiveTransform(base_coor, base_circle)

  print("transformation_matrix : " , transformation_matrix)
  result = cv2.warpPerspective(img, transformation_matrix,(baseline.shape[1], baseline.shape[0]))

  cv2_imshow(result)

  defensor_feet = []
  for i in range(len(defense_list)) :
    defensor_feet.append([defense_list[i][0] + (defense_list[i][2] - defense_list[i][0])/2,defense_list[i][3]])

  defensor_feet = np.array(defensor_feet)
  transformed_point = cv2.perspectiveTransform(defensor_feet.reshape(1, -1, 2), transformation_matrix)
  transformed_defensor_coor = transformed_point[0]

  real_coor = []
  for i in range(len(transformed_defensor_coor)) :
    bool_, cor = real_defense_coordinate(transformed_defensor_coor[i])
    if bool_ == True :
      real_coor.append(cor)

  return True, result, real_coor

In [ ]:
cor_dict = {}

In [ ]:
png_files = [image for image in os.listdir() if image.endswith('.png')]

for i, png_file in enumerate(png_files) :

  cnt = cv2.imread(path + f'image/{png_file}')
  bool_ , result, real_coor = main(cnt, 450,baseline,base_circle)
  if (bool_ == True) :
    if (len(real_coor) == 4):
      cor_dict[png_file] = real_coor


In [ ]:
cor_dict

{'2_0.png': [[-18.961331664668464, 28.428030206581628],
  [13.155353368300101, 47.03089229733295],
  [21.559438055172862, 29.220173057478156],
  [-9.89258942794539, 46.13444769939124]],
 '5_1.png': [[-15.378908193679173, 38.94547851192908],
  [-20.96632739293075, 26.79751009091108],
  [-1.72467893278933, 44.06141307732499],
  [19.209598001745363, 28.97060264011808]],
 '6_0.png': [[23.6750032783945, 27.06516619131224],
  [-11.835100439821876, 46.594789061890566],
  [12.027530599994659, 51.03455985317512],
  [-21.887832206097297, 27.211322809575922]],
 '10_0.png': [[-12.18582137198373, 48.563091156600066],
  [-21.85685341906878, 27.38329127414061],
  [4.998419063453805, 47.092406661444926],
  [18.64235505225307, 26.530605628113605]],
 '9_1.png': [[19.355052769273378, 19.998156051106136],
  [18.332421021454657, 43.458682816713264],
  [7.2108455031121155, 51.28726292843148],
  [-12.75998319500008, 28.796057321281076]],
 '12_0.png': [[17.69416262488611, 19.670291564819713],
  [-7.6935851846

In [ ]:
sorted_cor = {key: sorted(value, key=lambda x: x[0]) for key, value in cor_dict.items()}

In [ ]:
#coordinate_ = {}

In [ ]:
#for key,item in sorted_cor.items() :
#  newlist = [item[0], item[1], item[3], item[2]]
#  coordinate_[key] = newlist

In [ ]:
#coordinate_

In [ ]:
import json

with open(path + 'data.json', 'w') as json_file:

    json.dump(sorted_cor, json_file)